# 对情感分类进行特征提取

情感分类按照论文《情感词汇本体的构造》所述，情感分为7大类21小类。

情感分类：

- Positive = Happy + Good + Surprise

- Negative = Anger + Sad + Fear + Disgust

In [3]:
import pandas as pd
import jieba
import time
#-------------------------------------获取数据集---------------------------------
f=open('庆余年220.csv',encoding='utf8')
weibo_df = pd.read_csv(f)
print(weibo_df.head())

       id        time                                             review
0     红骷髅  2019/11/26                            达康书记的脸配上陈萍萍的名字，有种莫名的喜感。
1   黑色星期五  2019/11/26            这剧真是超豪华阵容呀，想看陈道明和张若昀的父子大戏，很喜欢达康书记演的陈萍萍。
2   沫沫在流浪  2019/11/26                                              过审鬼才。
3  Egorka  2019/11/26                          服道化low如于妈制作的雷剧 张若昀也是丑的没眼看
4   红色高跟鞋  2019/11/26  终于开播了，就冲这演员阵容不看真的可惜，陈道明，吴刚……虽然我没看过小说但超级期待，听说郭麒...


In [4]:
#-------------------------------------情感词典读取-------------------------------
#注意：
#1.词典中怒的标记(NA)识别不出被当作空值,情感分类列中的NA都给替换成NAU
#2.大连理工词典中有情感分类的辅助标注(有NA),故把情感分类列改好再替换原词典中

# 扩展前的词典
df = pd.read_excel('大连理工大学中文情感词汇本体NAU.xlsx')
print(df.head(10))

df = df[['词语', '词性种类', '词义数', '词义序号', '情感分类', '强度', '极性']]
df.head()


    词语  词性种类  词义数  词义序号 情感分类  强度  极性 辅助情感分类  强度.1  极性.1 Unnamed: 10  \
0   脏乱   adj  1.0   1.0   NN   7   2    NaN   NaN   NaN         NaN   
1   糟报   adj  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
2   早衰   adj  1.0   1.0   NE   5   2    NaN   NaN   NaN         NaN   
3   责备  verb  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
4   贼眼  noun  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
5   战祸  noun  1.0   1.0   ND   5   2     NC   5.0   2.0         NaN   
6   招灾   adj  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
7   折辱  noun  1.0   1.0   NE   5   2     NN   5.0   2.0         NaN   
8  中山狼  noun  1.0   1.0   NN   5   2    NaN   NaN   NaN         NaN   
9   清峻   adj  1.0   1.0   PH   5   0    NaN   NaN   NaN         NaN   

   Unnamed: 11  
0          NaN  
1          NaN  
2          NaN  
3          NaN  
4          NaN  
5          NaN  
6          NaN  
7          NaN  
8          NaN  
9          NaN  


,词语,词性种类,词义数,词义序号,情感分类,强度,极性
0,脏乱,adj,1.0,1.0,NN,7,2
1,糟报,adj,1.0,1.0,NN,5,2
2,早衰,adj,1.0,1.0,NE,5,2
3,责备,verb,1.0,1.0,NN,5,2
4,贼眼,noun,1.0,1.0,NN,5,2


In [5]:
#-------------------------------------七种情绪的运用-------------------------------
Happy = []
Good = []
Surprise = []
Anger = []
Sad = []
Fear = []
Disgust = []

#df.iterrows()功能是迭代遍历每一行
for idx, row in df.iterrows():
    if row['情感分类'] in ['PA', 'PE']:
        Happy.append(row['词语'])
    if row['情感分类'] in ['PD', 'PH', 'PG', 'PB', 'PK']:
        Good.append(row['词语']) 
    if row['情感分类'] in ['PC']:
        Surprise.append(row['词语'])       
    if row['情感分类'] in ['NB', 'NJ', 'NH', 'PF']:
        Sad.append(row['词语'])
    if row['情感分类'] in ['NI', 'NC', 'NG']:
        Fear.append(row['词语'])
    if row['情感分类'] in ['NE', 'ND', 'NN', 'NK', 'NL']:
        Disgust.append(row['词语'])
    if row['情感分类'] in ['NAU']:     #修改: 原NA算出来没结果
        Anger.append(row['词语'])  

#正负计算不是很准 自己可以制定规则       
Positive = Happy + Good + Surprise
Negative = Anger + Sad + Fear + Disgust
print('情绪词语列表整理完成')  
print(Anger)

情绪词语列表整理完成
['忿忿不平', '怒火冲天', '气愤愤', '失落', '愤懑', '悲愤', '勃然大怒', '含怒', '恼羞成怒', '忿怒', '七窍生烟', '发标', '怒容', '吹胡子', '唱老生的生气－吹胡子瞪眼镜', '暴跳如雷', '戟指怒目', '怒形于色', '怒目切齿', '敢怒不敢言', '除恶务尽', '逼上梁山', '阴沉', '急眼', '看扁', '气鼓鼓', '投诉', '令人生气', '没好气', '民怨沸腾', '悲愤交加', '大动肝火', '火气', '怒不可遏', '惹气', '窝火', '光火', '无明火', '耍态度', '触怒', '怒色', '犯肝气', '吃了芋头不下肚－顶心顶肺', '勃然变色', '雷霆之怒', '拍案而起', '切齿愤盈', '我靠', 'kao', 'gun', '吵嘴', '羞愤', '气急败坏', '嗟来之食', '火暴', '气忿', '无名火起', '愤愤不平', '暴怒', '愤怒', '恼火', '气冲冲', '义愤', '挂气', '愤愤', '私愤', '赌气', '横眉', '一刀两段', '一气一个死', '火箭筒射击－两头冒火', '怫然作色', '怒气冲冲', '义愤填膺', '上等轮胎－有气难出', '睚眦', '可气', '当场出彩', '针锋相对', 'TNND', '滚出去', '哇靠', '郁怒', '疾言厉色', '火冒三丈', '义形于色', '忿忿', '抗议', '面有愠色', '气忿忿', '割席断交', '天怒人怨', '悲愤填膺', '郁愤', '纳鞋底戳了手－真气人', '愤世嫉俗', '愤然', '恼怒', '气愤', '震怒', '动肝火', '悻然', '负气', '横眉怒目', '奇耻大辱', '草场着火－火气冲天', '怒气冲天', '气呼呼', 'MD', '真TM', '你丫', '嗔怒', '叱呵', '不甘示弱', '寒碜', '抱不平', '来气', '气汹汹', '竟敢', '愤激', '气不忿', '忧愤', '愤慨', '民愤', '怒视', '易怒', '动气', '大发雷霆', '公愤', '泄愤', '怒目', '一刀两断', '三尸暴跳，七窍生烟', '皇上拍桌子－盛怒', '艴然不悦'

In [6]:
#---------------------------------------中文分词---------------------------------

#添加自定义词典和停用词
#jieba.load_userdict("user_dict.txt")
stop_list = pd.read_csv('stop_words.txt',
                        engine='python',
                        encoding='utf-8',
                        delimiter="\n",
                        names=['t'])

#获取重命名t列的值
stop_list = stop_list['t'].tolist()
print(stop_list)
def txt_cut(juzi):
    return [w for w in jieba.lcut(juzi) if w not in stop_list]     #可增加len(w)>1


['&nbsp', '&nbsp;', 'aboard', 'about', 'above', 'according', 'according to', 'across', 'afore', 'after', 'afterwards', 'again', 'against', 'agin', 'all', 'almost', 'alone', 'along', 'alongside', 'already', 'also', 'although', 'always', 'am', 'amid', 'amidst', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'anent', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'approximately', 'are', 'around', 'as', 'asked', 'aslant', 'astride', 'at', 'athwart', 'back', 'bar', 'be', 'became', 'because', 'because of', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beneath', 'beside', 'besides', 'between', 'betwixt', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'called', 'can', 'cannot', 'cant', 'circa', 'co', 'computer', 'con', 'could', 'couldnt', 'cry', 'currently', 'dare', 'de', 'describe', 'despite', 'detail', 'did', 'do', 'does', 'done', 'down', 'dr', 'due', 'due to', 'during', 'e.g.,', 'each', 'earlier', 'eg

In [7]:
#---------------------------------------情感计算---------------------------------
def emotion_caculate(text):
    positive = 0
    negative = 0
    
    anger = 0
    disgust = 0
    fear = 0
    sad = 0
    surprise = 0
    good = 0
    happy = 0

    anger_list = []
    disgust_list = []
    fear_list = []
    sad_list = []
    surprise_list = []
    good_list = []
    happy_list = []
    
    wordlist = txt_cut(text)
    #wordlist = jieba.lcut(text)
    wordset = set(wordlist)
    wordfreq = []
    sort(wordset)
    print(wordset)
    for word in wordset:
        freq = wordlist.count(word)
        if word in Positive:
            positive+=freq
        if word in Negative:
            negative+=freq
        if word in Anger:
            anger+=freq
            anger_list.append(word)
        if word in Disgust:
            disgust+=freq
            disgust_list.append(word)
        if word in Fear:
            fear+=freq
            fear_list.append(word)
        if word in Sad:
            sad+=freq
            sad_list.append(word)
        if word in Surprise:
            surprise+=freq
            surprise_list.append(word)
        if word in Good:
            good+=freq
            good_list.append(word)
        if word in Happy:
            happy+=freq
            happy_list.append(word)
            
    emotion_info = {
        'length':len(wordlist),
        'positive': positive,
        'negative': negative,
        'anger': anger,
        'disgust': disgust,
        'fear':fear,
        'good':good,
        'sadness':sad,
        'surprise':surprise,
        'happy':happy,
        
    }

    indexs = ['length', 'positive', 'negative', 'anger', 'disgust','fear','sadness','surprise', 'good', 'happy']
    #return pd.Series(emotion_info, index=indexs), anger_list, disgust_list, fear_list, sad_list, surprise_list, good_list, happy_list
    return pd.Series(emotion_info, index=indexs)

#测试 (res, anger_list, disgust_list, fear_list, sad_list, surprise_list, good_list, happy_list)
text = """
原著的确更吸引编剧读下去，所以跟《诛仙》系列明显感觉到编剧只看过故事大纲比，这个剧的编剧完整阅读过小说。
配乐活泼俏皮，除了强硬穿越的台词轻微尴尬，最应该尴尬的感情戏反而入戏，故意模糊了陈萍萍的太监身份、太子跟长公主的暧昧关系，
整体观影感受极好，很期待第二季拍大东山之役。玩弄人心的阴谋阳谋都不狗血，架空的设定能摆脱历史背景，服装道具能有更自由的发挥空间，
特别喜欢庆帝的闺房。以后还是少看国产剧，太长了，还是精短美剧更适合休闲，追这个太累。王启年真是太可爱了。
"""
#res, anger, disgust, fear, sad, surprise, good, happy = emotion_caculate(text)
res = emotion_caculate(text)
print(res)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HP\AppData\Local\Temp\jieba.cache
Loading model cost 0.597 seconds.
Prefix dict has been built successfully.


{'闺房', '喜欢', '自由', '极好', '看过', '编剧', '原著', '台词', '戏', '吸引', '东山', '明显', '跟长', '感受', '发挥', '拍大', '狗血', '特别', '太子', '摆脱', '人心', '活泼', '模糊', '国产', '陈萍萍', '故意', '身份', '适合', '太长', '强硬', '服装道具', '王启年', '应该', '太累', '感觉', '休闲', '尴尬', '完整', '架空', '\n', '阳谋', '轻微', '故事', '大纲', '庆帝', '精短', '下去', '设定', '真是太', '可爱', '穿越', '系列', '小说', '暧昧关系', '阅读', '以后', '俏皮', '影', '剧', '入戏', '第二季', '追', '整体观', '配乐', '太监', '期待', '阴谋', '读', '空间', '更', '之役', '美剧', '历史背景', '玩弄', '少看', '公主', '诛仙', '感情'}
length      88
positive     7
negative     6
anger        0
disgust      6
fear         0
sadness      0
surprise     0
good         6
happy        1
dtype: int64


In [8]:
#---------------------------------------情感计算---------------------------------
start = time.time()   
emotion_df = weibo_df['review'].apply(emotion_caculate)
end = time.time()
print(end-start)
print(emotion_df.head())

#输出结果
output_df = pd.concat([weibo_df, emotion_df], axis=1)
output_df.to_csv('庆余年220_emotion.csv',encoding='utf_8_sig', index=False)
print(output_df.head())

#显示fear、negative数据集
fear_content = output_df.sort_values(by='fear',ascending=False)
print(fear_content)
print(fear_content.iloc[0:5]['review'])

negative_content = output_df.sort_values(by='negative',ascending=False)
print(negative_content)
print(negative_content.iloc[0:5]['review'])


{'名字', '书记', '脸配', '莫名', '陈萍萍', '达康', '有种', '喜感'}
{'真是', '阵容', '豪华', '想', '陈道明', '大戏', '喜欢', '书记', '超', '父子', '陈萍萍', '演', '这剧', '达康', '张若昀'}
{'鬼才', '过审'}
{'服道化', '张若昀', '没', '制作', '如于', ' ', '雷剧', '眼看', '妈', '也', '丑', 'low'}
{'看过', '听说', '演员阵容', '小说', '郭', '观众', '不看', '希望', '剧情', '嗑', '开播', '超级', '麒麟', '也', '真的', '追', '吴刚', '可惜', '陈道明', '没', '超', '超美', '一点', '终于', '搞笑', '期待', '李沁'}
{'关注度', '喜欢', '史上', '庆帝', '不错', '认可', '更', '编剧', '整体', '强大', '阵容', '大大', '卡司', '小说', '表现', '没有', '感觉', '庆', '关注', '演会', '张若昀', '希望', '毕竟', '剧', '恩恩', '最大', '理由', '也', '曾经', '会', '当年', '惊喜', '陈道明', '莫名', '太高', '余年', '超', '带来', '威武', '反派', '王倦', '前段时间', '这部'}
{'不准', '不得不', '喜欢', '小范', '接受', '追书', '不要', '有生之年', '腾讯', 'ps', '电视剧', '系列', '可能', '开头', '希望', '知道', '拍成', '算是', '部分', '色', '长大', '时', '不少', '剧增', '多年', '追', '上学', '改编', '说', '快', '一会儿', '天昏地暗', '终于', '闲给'}
{'吴刚', '后面', '好入', '挺', '精彩', '意思', '戏', '陈萍萍', '太酷', '第一集', '感觉', '造型', '会', '看着'}
{'喜欢', '带', '小范', '很服', '没得说', '第一集', '度', '设定', '真心', '起来', '还原',

{'黑手', '身边', '怎么', '不得不', '有点', '厉害', '遗志', '故事', '全篇', '过去', '看过', '主角', '原著', '实现', '总共', '权力', '十来', '穿越', '86', '开挂', '小说', '游戏', '默写', '更新', '林婉儿', '第二集', '家丁', '就会造', '一周', '弄死', '差不多', ' ', '估计', '结婚', '整个', '年', '统一', '也', '最后', '原子弹', '第一季', '奇遇', 'yy', '肯定', '一下', '明白', '陈道明', '全人类', '4', '没', '红楼梦', '极品', '说', '拍拍', '天', '集', '闹', '诛仙', '46', '什么', '美女', '母亲'}
{'琅', '延禧', '带', '苏了', '杰克', '皇后', '7', '不要', '玛丽', '主角', '8', '攻略', '脑子', '继', '哪个', '太尬', '喜剧', '女主', '笑点', '立体', '分', '脸谱化', '人物', '太', '沉稳', '看不下去', '塑造', '榜', '龙傲', '剧情', '苏大剧', '角色', '男主', '严重', '大气', '真的', '琊', '好歹', '说', '去年', '天', '里', '这部', '漏洞百出', '包括'}
{'老', '拿枪', '艺术家', '调', '某位'}
{'啥时候', '剧', '类型', '突破', '不行'}
{'怎么', '揍', '合拍片', '垃圾', '一问', '设定', '这天', '大大', '挨', '避开', '气质', '性', '一刷', '港台', '悬浮', '审', '演员', '也', '中', '灵魂', '掺合', '没', '二', '基本', '王倦', '一脚', '地上'}
{'记', '升职', '预算', '服化', '充足', '太子妃', '道'}
{'故事', '杰克', '暗示', '作品', '铺天盖地', '现在', '社交', '内核', '吹', '吹捧', '未', '开播', '六集', '平台', '成', '完', '失望', '

{'死', '数', '不了', '不用', '几十分钟', '接受', '百遍', '主角', '凑集', '13', '团队', '或前', '憨', '不能', '慢', '内容', '穿插', '两', '早就', '回忆', '真是', '奔', ' ', '也', '爽剧', '会', '光环', '架空', '改编', '现代', '节奏', '极其', '鬼才', '预告', '时不时', '播过', '集', '爽文', '基本', '古代', '一集', '一爽文', '前', ',', '时代'}
{'老师', '历史', '宣传', '支持', '想起', '疯狂', '不行', '原著', '歪', '轶', '买单', '言论', '一个', '人品', '聚集地', '太丑', '最好', '值得', '保护色', '糊是', '黑子', '美男', '男主', '霍去病', '公然', '演技', '偶像', '乐天', '简直', '女星', '一星', '也', '不可', '绝世', '陈道明', '歪曲', '发表', '劝退', '面前', '弹幕', '李沁宋', '还演', '这剧', '作者', '王倦', '国家'}
{'怎么', '知道', '鬼', '这种', '剧本', '完全', '刻意', '什么', '拿到', '无厘头', '投资', '干嘛', '搞笑', '两集', '很尬'}
{'一文不值', '一派胡言'}
{'觉得', '再', '既视感', '独步天下', '好看', '唐艺昕', '两集', '看看'}
{'怎么', '正剧', '横', '男主太', '适合', '胖', '主要', '太', '古装', '身材', '演技', '友好', '雷剧', '也', '反复', '中', '猪', '两种', '浮夸', '风格', '长', '电视观众', '大饼', '跳', '像头', '脸', '搞笑', '丑'}
{'说实话'}
{'未晚', '营养', '再', '呼风唤雨', '阶级', '领导', '官宦', '利益', '表现', '实则', '世家', '高于', '从小', '中', '一处', '太子党', '博弈', '无视', '一点', '十年', '悲剧', 

{'死', '笑', '哪里', '哈哈哈', '剧', '看病', '好久没', '追过', '林宛儿'}
{'可爱', '麒麟', '郭', '真的'}
3.760995864868164
   length  positive  negative  anger  disgust  fear  sadness  surprise  good  \
0       8         0         0      0        0     0        0         0     0   
1      15         1         0      0        0     0        0         0     1   
2       2         0         0      0        0     0        0         0     0   
3      12         0         0      0        0     0        0         0     0   
4      28         1         1      0        0     0        1         0     1   

   happy  
0      0  
1      0  
2      0  
3      0  
4      0  
       id        time                                             review  \
0     红骷髅  2019/11/26                            达康书记的脸配上陈萍萍的名字，有种莫名的喜感。   
1   黑色星期五  2019/11/26            这剧真是超豪华阵容呀，想看陈道明和张若昀的父子大戏，很喜欢达康书记演的陈萍萍。   
2   沫沫在流浪  2019/11/26                                              过审鬼才。   
3  Egorka  2019/11/26                          服道化lo